In [90]:
#Script para gerar tabela Forma Fornecedor

import pandas as pd
from sqlalchemy import create_engine, VARCHAR,NUMERIC,INTEGER,DATE,DATETIME,String,text,CHAR
import random
import pyodbc
from datetime import date
import re

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
engine = create_engine(conexaostring)

#Função para testar a conexão
def testar_conexao_sqlalchemy(engine):
    try:        
        with engine.connect() as conexao: #Tenta conectar ao banco de dados
            print("Conexão com o banco de dados estabelecida com sucesso")
            result = conexao.execute(text("SELECT 1 AS teste")) #Executa uma consulta para testar a conexão
            print("Query executada com sucesso")
            return True
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return False
#testa a conexão
if testar_conexao_sqlalchemy(engine): 
    print("A conexão foi bem-sucedida")
else:
    print("Falha na conexão.")

nomes_ruas = [
    "Rua Sao Joao", "Rua Santa Maria", "Rua Sao Paulo", "Rua Sao Pedro", "Rua Sao Jose",
    "Rua Sao Francisco", "Rua Santo Antonio", "Rua Sao Sebastiao", "Rua Santa Catarina",
    "Rua Luzia", "Rua Santa Rita", "Rua Santa Ana", "Rua Santa Terezinha", "Rua Santa Clara",
    "Rua Isabel", "Rua Santa Helena", "Rua Santa Cecilia", "Rua Santa Rosa", "Rua Santa Monica",
    "Rua Edwiges", "Rua Santa Ines", "Rua Santa Fe", "Rua Santa Lucia", "Rua Santa Efigenia",
    "Rua Barbara", "Rua Santa Angela", "Rua Santa Gertrudes", "Rua Santa Margarida",
    "Rua Marina", "Rua Santa Elisa", "Rua Santa Julia", "Rua Santa Zelia", "Rua Santa Zilda",
    "Rua Zulmira", "Rua Santa Zulene, Rua das Flores", "Rua da Paz", "Rua Sao Joao", "Rua Sao Paulo", "Rua Santo Antonio",
    "Rua 7 de Setembro", "Rua XV de Novembro", "Rua do Comercio", "Rua das Acacias", "Rua das Palmeiras",
    "Rua dos Cravos", "Rua do Sol", "Rua Lua Nova", "Rua da Esperanca", "Rua Sao Pedro",
    "Rua das Orquideas", "Rua das Rosas", "Rua Primavera", "Rua Santa Luzia", "Rua Sao Francisco",
    "Rua das Margaridas", "Rua do Bosque", "Rua das Aguas", "Rua Bela Vista", "Rua Santa Catarina",
    "Rua Dom Pedro I", "Rua Dom Pedro II", "Rua Marechal Deodoro", "Rua da Liberdade", "Rua do Lago",
    "Rua dos Girassois", "Rua Antonio Carlos", "Rua da Independencia", "Rua Presidente Vargas", "Rua das Laranjeiras",
    "Rua do Porto", "Rua Santos Dumont", "Rua Carlos Gomes", "Rua da Vitoria", "Rua do Carmo",
    "Rua do Campo", "Rua da Saudade", "Rua do Progresso", "Rua dos Bandeirantes", "Rua Coronel Antonio",
    "Rua Sao Jose", "Rua Benedito Calixto", "Rua dos Lirios", "Rua Sete Lagoas", "Rua Pedro Alvares Cabral",
    "Rua Castro Alves", "Rua Padre Anchieta", "Rua Machado de Assis", "Rua Monte Castelo", "Rua Humberto de Campos",
    "Rua dos Tucanos", "Rua das Violetas", "Rua do Ipe", "Rua dos Pinheiros", "Rua Jose Bonifacio",
    "Rua Getulio Vargas", "Rua Marechal Rondon", "Rua Visconde de Maua", "Rua do Horizonte", "Rua do Contorno",
    "Rua Rui Barbosa", "Rua Juscelino Kubitschek", "Rua Presidente Joao Goulart", "Rua General Osorio", "Rua dos Sonhos",
    "Rua das Figueiras", "Rua Goias", "Rua Alvorada", "Rua Bela Cintra", "Rua Piaui",
    "Rua Domingos Jorge Velho", "Rua General Carneiro", "Rua Dom Bosco", "Rua Miguel Couto", "Rua Alfredo Pujol",
    "Rua Frei Caneca", "Rua Dr. Arnaldo", "Rua Diogo Feijo", "Rua Regente Feijo", "Rua Conselheiro Lafaiete",
    "Rua Barao de Maua", "Rua Benjamin Constant", "Rua Santa Clara", "Rua Maranhao", "Rua Bahia",
    "Rua Amazonas", "Rua Acre", "Rua Sao Joaquim", "Rua dos Cedros", "Rua Tamoios",
    "Rua Jose de Alencar", "Rua Ceara", "Rua Camelia", "Rua das Cerejeiras", "Rua Japao"]

nomes_bairros = [
    "Centro", "Jardim", "Vila", "Sao Joao", "Sao Paulo", "Sao Pedro", "Sao Jose", "Sao Francisco",
    "Santo Antonio", "Sao Sebastiao", "Santa Maria", "Santa Catarina", "Santa Luzia", "Santa Rita",
    "Santa Ana", "Santa Terezinha", "Santa Clara", "Santa Isabel", "Santa Helena", "Santa Cecilia",
    "Santa Rosa", "Santa Monica", "Santa Edwiges", "Santa Ines", "Santa Fe", "Santa Lucia",
    "Santa Efigenia", "Santa Barbara", "Santa Angela", "Centro", "Jardim das Flores", "Vila Nova", "Jardim America", "Jardim Paulista",
    "Jardim Primavera", "Jardim Europa", "Jardim dos Ipes", "Jardim das Palmeiras", "Jardim Bela Vista",
    "Parque Industrial", "Jardim Alvorada", "Jardim Santa Clara", "Vila Esperanca", "Vila Maria",
    "Parque Sao Jorge", "Jardim Sao Francisco", "Jardim Santa Rita", "Vila Rica", "Vila Progresso",
    "Vila Matilde", "Vila Boa Vista", "Vila Sao Jose", "Parque dos Passaros", "Jardim Imperial",
    "Jardim Sao Luiz", "Jardim das Acacias", "Jardim Mariana", "Jardim do Lago", "Parque das Nacoes",
    "Parque dos Bandeirantes", "Jardim Atlantico", "Jardim Monte Castelo", "Jardim Eldorado", "Jardim Gloria",
    "Vila Oliveira", "Jardim Panorama", "Jardim Dom Bosco", "Vila Uniao", "Jardim Vitoria",
    "Jardim Novo Horizonte", "Vila Industrial", "Vila Augusta", "Jardim Botanico", "Jardim Sao Carlos",
    "Jardim Canada", "Jardim Santa Fe", "Jardim das Hortensias", "Parque Sao Bento", "Jardim dos Pinheiros",
    "Parque do Carmo", "Jardim Sao Joao", "Jardim Maria Helena", "Jardim Santa Rosa", "Jardim Felicidade",
    "Jardim Aurora", "Vila Santana", "Jardim Bela Vista", "Jardim das Magnolias", "Vila Medeiros",
    "Jardim dos Lirios", "Parque das Arvores", "Jardim Santa Terezinha", "Vila Nova Esperanca", "Jardim Santo Antonio",
    "Jardim Independencia", "Jardim Sao Jorge", "Jardim das Oliveiras", "Parque Estrela Dalva", "Jardim Mirante",
    "Vila Sao Pedro", "Parque Bela Vista", "Vila Nova Cachoeirinha", "Jardim Monte Alegre", "Parque Imperial",
    "Jardim Tropical", "Vila Santa Isabel", "Jardim Italia", "Jardim dos Cedros", "Jardim das Nacoes",
    "Vila Real", "Jardim dos Jacarandas", "Jardim Primavera", "Parque Paraiso", "Jardim Universo",
    "Jardim Santo Andre", "Jardim das Figueiras", "Vila Germanica", "Jardim Luz", "Vila Macedo",
    "Jardim Bonfim", "Jardim Colonial", "Jardim das Tulipas", "Vila Curuca", "Jardim Santa Maria"
]

lista_forma_FORNECEDORES = ['Teixeira  EI', 'Monteiro Ltda', 'Cunha', 'Freitas  EI', 'Brito da Cunha Ltda', 'Azevedo SA', 'Porto Sousa SA', 'Leao', 'da Rosa', 'Barbosa', 'da Cunha Cardoso e Filhos', 'Duarte', 'Sa', 'Mendonca', 'Pereira Fonseca SA', 'Pires', 'Vargas', 'Campos', 'Pires', 'da Mata Casa Grande SA', 'Caldeira', 'Camara e Filhos', 'Pereira  ME', 'Guerra SA', 'Cassiano Ltda', 'Caldeira SA', 'Costa', 'Azevedo SA', 'Pastor Melo SA', 'Monteiro', 'das Neves Pinto SA', 'Campos', 'Macedo', 'Pereira Cunha  ME', 'Rodrigues', 'Araujo', 'Sa', 'Oliveira', 'Cassiano', 'Farias', 'Novais', 'Ribeiro', 'Novais Rodrigues  EI', 'Cavalcante da Rosa SA', 'Pacheco SA', 'Rodrigues', 'Gomes', 'Monteiro', 'Moraes', 'Cassiano', 'Fernandes', 'Albuquerque', 'Carvalho', 'Cirino Casa Grande  ME', 'Barros SA', 'Lima', 'da Rosa', 'da Rocha Viana SA', 'da Mata Pereira e Filhos', 'Vieira Silva  EI', 'Macedo', 'da Rosa  ME', 'Sampaio', 'da Luz Cunha e Filhos', 'Camara', 'Pires Moura  ME', 'Abreu', 'da Rosa Ltda', 'Carvalho', 'Porto', 'Goncalves', 'da Conceicao Ribeiro e Filhos', 'Porto Ramos  EI', 'Montenegro', 'Mendes Brito Ltda', 'Caldeira Garcia e Filhos', 'Borges  ME', 'Araujo Farias  ME', 'da Cunha', 'Jesus e Filhos', 'Pacheco Ltda', 'Fernandes', 'Fonseca  ME', 'Cirino Ltda', 'Moraes', 'Brito', 'Fogaca Alves e Filhos', 'Santos', 'Silva Silva SA', 'Macedo SA', 'Sousa Goncalves Ltda', 'Araujo', 'Oliveira SA', 'Sa Ltda', 'Machado', 'Goncalves Freitas SA', 'Borges', 'da Rocha  ME', 'Correia', 'Barbosa da Paz SA', 'da Costa', 'Silveira SA', 'Cunha', 'Fernandes Lopes SA', 'da Luz', 'Correia SA', 'Freitas', 'da Cruz SA', 'da Paz Moura Ltda', 'Novaes', 'Caldeira', 'Oliveira', 'Viana', 'Borges', 'da Rosa', 'Cirino', 'Gomes  ME', 'da Costa Marques  EI', 'Sales', 'Mendonca', 'Sa', 'Guerra', 'Sa  EI', 'Cunha  EI', 'Freitas Ltda', 'Caldeira Vasconcelos e Filhos', 'das Neves e Filhos', 'Andrade', 'Moreira Vieira SA', 'Rezende SA', 'Borges Marques SA', 'Lopes Ltda', 'Barros Farias SA', 'Garcia', 'Araujo', 'Lopes Ramos  EI', 'Rodrigues  ME', 'Marques', 'Castro  EI', 'Mendonca Andrade  ME', 'Porto Ribeiro SA', 'Vasconcelos', 'Dias SA', 'Camargo', 'Carvalho', 'Pinto', 'Mendonca', 'Dias', 'Martins SA', 'Peixoto Lopes  ME', 'Rocha', 'Rezende', 'da Conceicao', 'Gomes Carvalho  ME', 'da Mata Aparecida  EI', 'Abreu', 'da Rocha  EI', 'Nunes Novais SA', 'Carvalho', 'Nascimento', 'Peixoto Rios SA', 'Borges', 'da Luz  ME', 'Santos', 'Lima', 'Cavalcante  ME', 'Jesus', 'Nascimento', 'Silveira Moraes SA', 'Fernandes Fernandes  ME', 'Castro', 'Viana  EI', 'Rocha', 'Brito SA', 'Macedo Ltda', 'Pinto e Filhos', 'Araujo', 'Cirino', 'Barbosa', 'Cirino da Paz SA', 'Castro', 'Macedo', 'Silva', 'Gomes', 'Ferreira', 'Fogaca', 'Silva', 'da Cunha', 'Moraes', 'Vasconcelos  ME', 'Caldeira da Luz  EI', 'Castro Camara Ltda', 'Almeida', 'Rezende  ME', 'Cavalcanti SA', 'Abreu', 'Farias Cassiano  ME', 'Pinto', 'Carvalho Almeida SA', 'Sousa da Conceicao Ltda', 'Peixoto da Luz  ME', 'Azevedo', 'Oliveira', 'Borges e Filhos', 'Pacheco Jesus  EI', 'Moura', 'Cavalcanti Carvalho SA', 'Sampaio', 'Farias Freitas SA', 'Barbosa', 'da Cunha', 'Albuquerque', 'Cavalcanti Porto Ltda', 'Ribeiro  EI', 'Aparecida', 'Jesus SA', 'Monteiro', 'Pires', 'Almeida Ramos SA', 'Peixoto', 'Fernandes Moura Ltda', 'Garcia', 'Brito', 'das Neves Ltda', 'Castro', 'Castro e Filhos', 'Moura', 'Nogueira', 'Oliveira', 'Mendonca', 'Abreu Melo Ltda', 'Santos', 'Cirino Ltda', 'Ferreira', 'da Mota', 'da Mata', 'Nascimento  ME', 'da Paz SA', 'Moraes e Filhos', 'Martins', 'Sampaio', 'Caldeira  ME', 'Montenegro', 'Montenegro  EI', 'Sa Silveira SA', 'Martins e Filhos', 'Mendes Nogueira  ME', 'Viana Mendes Ltda', 'Cavalcante', 'Nascimento SA']

lista_prefixos_empresas = ['Distribuidora','Dist','Comercial','Atacado', 'Atacadao']

cidades_brasil = [ "Sao Paulo", "Rio de Janeiro", "Brasilia", "Salvador", "Fortaleza",
    "Belo Horizonte", "Curitiba", "Recife", "Goiania", "Porto Alegre", "Guarulhos", 
    "Campinas", "Sao Bernardo do Campo", "Ribeirao Preto", "Uberlandia", "Contagem", "Sorocaba",
    "Cuiaba", "Joinville", "Londrina", "Juiz de Fora", "Maua", "Piracicaba", "Sao Jose dos Campos"] 

provedores = ["gmail.com", "yahoo.com", "hotmail.com", "outlook.com"]

df_maiores_cidades = df = pd.read_sql("SELECT IDCIDADE FROM CIDADE WHERE NOME IN ({})".format(','.join(f"'{cidade}'" for cidade in cidades_brasil)), engine)['IDCIDADE'].tolist()

lista_id_produto = pd.read_sql("SELECT IDPRODUTO FROM PRODUTO", engine)['IDPRODUTO'].tolist()

quant_fornecedores_gerados = 10000

# Função para gerar um nome completo, gênero e e-mail
def gerar_fornecedor_completo():
    prefixo = random.choice(lista_prefixos_empresas)
    nomecompleto = f"{prefixo} {random.choice(lista_forma_FORNECEDORES)}"
    nome = re.sub(r'[^a-zA-Z0-9]', ' ', nomecompleto)
    #sobrenome = random.choice(sobrenomes)
    #sobrenome2 = random.choice(sobrenomes)
    #sobrenome3 = random.choice(sobrenomes)
    #nomecompleto = f'{nome} {sobrenome} {sobrenome2} {sobrenome3}'
    #genero = "F" if nome in nomes_femininos else "M"
    #numero = random.randint(10, 9999)
    #usuario = f"{nome.lower()}{sobrenome.lower()}{sobrenome2.lower()}{numero}"
    
    email = f"{(re.sub(r'[^a-zA-Z0-9]', '', nome.replace(" ", ""))).lower()}@{(re.sub(r'[^a-zA-Z0-9]', '', nome.replace(" ", ""))).lower()}.com"  # Formatar o e-mail
    rua = random.choice(nomes_ruas)
    num = random.randint(5, 5000)
    bairro = random.choice(nomes_bairros)
    endereco = f"{rua} - {num} - {bairro}"
    id_cidade = random.choice(df_maiores_cidades)
    #id_produto = random.choice(lista_id_produto)
    return nome, email, endereco, id_cidade

# Gerar 1000 nomes completos com gênero e e-mail
dados_fornecedores = [gerar_fornecedor_completo() for _ in range(quant_fornecedores_gerados)]
df_fornecedores = pd.DataFrame(dados_fornecedores, columns=['NOME','EMAIL', 'ENDERECO', 'ID_CIDADE'])

def gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade):
    if quantidade > (intervalo_fim - intervalo_inicio + 1):
        raise ValueError("A quantidade de números únicos solicitada excede o intervalo disponível.")
    numeros_unicos = random.sample(range(intervalo_inicio, intervalo_fim + 1), quantidade)
    return numeros_unicos
#Valores de entrada
intervalo_inicio = 10000000000000  # Início do intervalo
intervalo_fim =    99999999999999  # Fim do intervalo
quantidade_registros = quant_fornecedores_gerados  # Quantidade de registros no DataFrame

lista_CNPJ = gerar_numeros_unicos(intervalo_inicio, intervalo_fim, quantidade_registros)

intervalo_inicio_tel = 980000000  
intervalo_fim_tel =    999999999
lista_ddd = [''.join([str(random.randint(1, 9)) for _ in range(2)]) for _ in range(quant_fornecedores_gerados)] 
lista_tel = gerar_numeros_unicos(intervalo_inicio_tel, intervalo_fim_tel, quant_fornecedores_gerados) 
lista_tel_conc = [item1 + str(item2) for item1, item2 in zip(lista_ddd, lista_tel)] #concatena os itens da lista_ddd e da lista_tel
lista_telefone = [f"({str(numero)[:2]}) {str(numero)[2:7]}-{str(numero)[7:]}" for numero in lista_tel_conc]

df_fornecedores["CNPJ"] = lista_CNPJ
df_fornecedores["TELEFONE"] = lista_telefone

#Tratamento no dados de entrada
df_fornecedores = df_fornecedores.drop_duplicates(subset=['EMAIL'])
df_fornecedores = df_fornecedores.apply(lambda col: col.str.replace(r'\s+', ' ', regex=True).str.strip() if col.dtype == 'object' else col) #remove espacos antes e depois de uma string


df_fornecedor= pd.DataFrame({
        
                "NOME":          df_fornecedores['NOME'],
                "CNPJ":          df_fornecedores['CNPJ'],
                "TELEFONE":      df_fornecedores['TELEFONE'],
                "EMAIL":         df_fornecedores['EMAIL'],
                "ENDERECO":      df_fornecedores['ENDERECO'],
                "ID_CIDADE":     df_fornecedores['ID_CIDADE'],
                "DATA_CRIACAO":  date.today().strftime('%d/%m/%Y'),
                "DATA_ATLZ":     "",
                "VERSAO":        1,
                
})

display(df_fornecedor)

tipo_dados_fornecedores = {
                
                "NOME":         VARCHAR(100),
                "CNPJ":         VARCHAR(14),
                "TELEFONE":     VARCHAR(20),
                "EMAIL":        VARCHAR(100),
                "ENDERECO":     VARCHAR(100),
                "ID_CIDADE":    INTEGER,
                "DATA_CRIACAO": DATE,
                "DATA_ATLZ":    DATE,
                "VERSAO":       INTEGER,
}

with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE FORNECEDOR"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

df_fornecedor.to_sql(name='FORNECEDOR',con=engine,if_exists='append',index=False,dtype=tipo_dados_fornecedores)

engine.dispose()

Conexão com o banco de dados estabelecida com sucesso
Query executada com sucesso
A conexão foi bem-sucedida


,NOME,CNPJ,TELEFONE,EMAIL,ENDERECO,ID_CIDADE,DATA_CRIACAO,DATA_ATLZ,VERSAO
0,Dist Sa Silveira SA,79843818982403,(88) 98417-5171,distsasilveirasa@distsasilveirasa.com,Rua Santa Maria - 619 - Jardim do Lago,3596,14/03/2025,,1
1,Atacadao Cassiano,22476240484580,(79) 98345-4897,atacadaocassiano@atacadaocassiano.com,Rua Santa Catarina - 2108 - Santa Catarina,3828,14/03/2025,,1
2,Distribuidora da Rosa Ltda,99771833969520,(29) 99067-0023,distribuidoradarosaltda@distribuidoradarosaltd...,Rua Dr. Arnaldo - 882 - Vila Curuca,4101,14/03/2025,,1
3,Atacadao Araujo,49323980567620,(49) 98392-3355,atacadaoaraujo@atacadaoaraujo.com,Rua Santa Zilda - 2765 - Jardim Sao Carlos,5214,14/03/2025,,1
4,Dist Cavalcante,61903372313288,(33) 98607-5591,distcavalcante@distcavalcante.com,Rua das Margaridas - 2659 - Parque do Carmo,2308,14/03/2025,,1
...,...,...,...,...,...,...,...,...,...
6024,Dist Vasconcelos,36607250799728,(15) 99877-3893,distvasconcelos@distvasconcelos.com,Rua das Cerejeiras - 887 - Santa Luzia,2447,14/03/2025,,1
6116,Dist Barros SA,88416954320222,(68) 98102-6809,distbarrossa@distbarrossa.com,Rua Santo Antonio - 2524 - Jardim Santa Terezinha,1595,14/03/2025,,1
6266,Dist Pacheco SA,77666289647671,(86) 98919-8708,distpachecosa@distpachecosa.com,Rua Lua Nova - 1577 - Santa Barbara,3478,14/03/2025,,1
6377,Distribuidora Goncalves,64520064280148,(84) 98441-3630,distribuidoragoncalves@distribuidoragoncalves.com,Rua Benedito Calixto - 1742 - Santa Cecilia,3699,14/03/2025,,1
